In [1]:
import pandas as pd
from geopy.distance import geodesic

In [32]:
viagens = pd.read_parquet("dados/trips_historico.parquet")
rotas = pd.read_parquet("dados/routes_historico.parquet")
passageiros=pd.read_parquet("dados\passageiros_unificado.parquet")
df_routes_categorizado=pd.read_parquet(r"dados\rotas_historico_categorizado.parquet")
df_freq_horarios=pd.read_parquet(r"dados\frequencies_horario_operacao_historico.parquet")

In [3]:
viagens['lat_lon']=list(zip(viagens['stop_lat'], viagens['stop_lon']))


distância entre o começo e ponto mais longe da rota

In [ ]:
import numpy as np
from scipy.spatial import ConvexHull, QhullError
from math import radians, sin, cos, sqrt, atan2

def haversine(coord1, coord2):
    R = 6371.0  # raio da Terra em km
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def max_geodesic_distance(coord_series):
    coords = np.array(list(coord_series))

    if len(coords) < 2:
        return 0.0
    if len(coords) == 2:
        return haversine(coords[0], coords[1])

    try:
        hull = ConvexHull(coords)
        hull_points = coords[hull.vertices]
    except QhullError:
        hull_points = coords

    max_dist = 0.0
    n = len(hull_points)
    for i in range(n):
        for j in range(i + 1, n):
            dist = haversine(hull_points[i], hull_points[j])
            if dist > max_dist:
                max_dist = dist
    return max_dist


In [ ]:
rotas_distkm = (
    viagens
    .groupby(['route_id','data_referencia'])
    .agg(
        dist_km=('lat_lon', max_geodesic_distance),
    )
    .reset_index()
   )

rotas_distkm.to_parquet(r"dados\routes_dist_km_historico.parquet",index=False)

In [ ]:
import pandas as pd
from geopy.distance import geodesic

# Exemplo: assumindo que seu dataframe se chama df
# Garantir ordenação correta para cada rota e data
rotas_dist_km_percorrido=viagens.sort_values(by=["route_id", "data_referencia", "trip_id", "stop_sequence"])

# Função para calcular distância total de uma rota
def calcular_distancia(grupo):
    coords = list(zip(grupo["stop_lat"], grupo["stop_lon"]))
    distancia = 0
    for i in range(len(coords)-1):
        distancia += geodesic(coords[i], coords[i+1]).meters
    return distancia

# Aplicar a função agrupando por rota e data
rotas_dist_km_percorrido=(
    rotas_dist_km_percorrido.groupby(["route_id", "data_referencia"])
      .apply(calcular_distancia)
      .reset_index(name="distancia_metros")
)

# Se preferir em km
rotas_dist_km_percorrido["distancia_km_percorrido"] = rotas_dist_km_percorrido["distancia_metros"] / 1000
rotas_dist_km_percorrido.to_parquet(r"dados\routes_dist_km_percorridos_historico.parquet",index=False)

Qtd de passageiros

In [33]:
df_pivot_passageiros = passageiros.pivot_table(index='data', columns='route_id', values='passageiros_total', aggfunc='sum')
df_pivot_passageiros = df_pivot_passageiros.fillna(0).sort_index()
df_pivot_passageiros.to_parquet(r"dados\passageiros_historico_pivot.parquet",index=True)

Freq. ideal de onibus

In [34]:
passageiros['dia_da_semana']=passageiros.data.dt.weekday
passageiros['mes']=passageiros.data.dt.month
passageiros['ano']=passageiros.data.dt.year

In [36]:
df_freq_horarios['data_referencia']=pd.to_datetime(df_freq_horarios["data_referencia"], format="%Y%m").dt.to_period("M")

TypeError: Passing PeriodDtype data is invalid. Use `data.to_timestamp()` instead

In [37]:
passageiros['data_referencia']=pd.to_datetime(passageiros["data"]).dt.to_period("M")

In [38]:
passageiros=passageiros.merge(df_freq_horarios[['horas_operacao','route_id','data_referencia']],on=['route_id','data_referencia'])
passageiros['veiculos_padron_necessarios']=passageiros['passageiros_total'] / 80
passageiros['veiculos_articulados_necessarios']=passageiros['passageiros_total'] / 160

passageiros['frequencia_ideal_padron']=passageiros['horas_operacao']*60 / passageiros['veiculos_padron_necessarios']
passageiros['frequencia_ideal_articulados']=passageiros['horas_operacao']*60 / passageiros['veiculos_articulados_necessarios']

In [39]:
df_freq_ideal=passageiros[['data', 'route_id',
        'dia_da_semana', 'mes', 'ano',
       'horas_operacao', 'veiculos_padron_necessarios',
       'veiculos_articulados_necessarios', 'frequencia_ideal_padron',
       'frequencia_ideal_articulados']]

In [40]:
passageiros.to_parquet(r"dados\passageiros_historico_frequencia_ideal.parquet",index=True)